### Scraper for <a href="https://fdsupply.org/archive/category/%E5%8D%B8%E5%A3%B2%E5%B8%82%E5%A0%B4%E3%81%AE%E8%AA%B2%E9%A1%8C"> Fdsupply </a> website

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os
import requests

In [2]:
def get_chrome_Driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    return driver

def get_soup(URL):
    r = requests.get(URL)
    if r:
        driver = get_chrome_Driver()
        driver.get(URL)
        htmlContent = driver.page_source
        soup =  BeautifulSoup(htmlContent, 'html.parser')
        driver.close()
        return soup
    else:
        print("Error Fetching Soup, Scraping Ended")
        return False

def get_link_from_soup(soup):

    link_list = list()
    try:
        wanted_div = soup.findAll("div", {"class":"archive-entries"})[0]
    except IndexError:
        print("HTML element not found, returning empty set")
        return set()
    for div in wanted_div.findAll("section", {"class":"archive-entry test-archive-entry autopagerize_page_element"}):
        wanted_article = div.findAll('a', href=True)[1]["href"]
        link_list.append(wanted_article)
    return link_list

def save_link_as_txt(link_list):
    file_path = "links/fdsupply.txt"
    # if file exist, get its content, merge to existing set and then overwrite
    if os.path.isfile(file_path):
        with open(file_path, 'r+', encoding="utf-8") as file:
            # get existing links in file
            old_links = set(file.readlines())
            # remove duplicates 
            link_list = link_list - old_links
            file.seek(0)
            for i in link_list:
                file.write(i+"\n")
            file.truncate()
    else: # if not, make new file
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, "w", encoding="utf-8") as file:
            for i in link_list:
                file.write(i+"\n")
    print(f"Links Saved in {file_path}")
    return True

In [3]:
all_links = set()
for i in range(1,100):
    soup = get_soup(f"https://fdsupply.org/archive/category/%E5%8D%B8%E5%A3%B2%E5%B8%82%E5%A0%B4%E3%81%AE%E8%AA%B2%E9%A1%8C?page={i}")
    if soup:
        current_links = get_link_from_soup(soup)
        if len(current_links) > 0: # continue until no tags are found 
            all_links.update(current_links)
        else: # if tags we want are not found
            break
    else: # if encountered HTML error such as 404
        break

save_link_as_txt(all_links)

Error Fetching Soup, Scraping Ended
Links Saved in links/fdsupply.txt


True